In [218]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv('Database-家榆的記帳表.csv')

print(df.head())

   Date  Breakfast  Lunch  Dinner  Dessert  Drink  Cat  Transport  \
0     1          0      0     125        0     15    0          0   
1     2        155      0     280       25      0    0          0   
2     3        270      0       0       53     40  425         76   
3     4        130      0     283       62      0    0          0   
4     5        165      0     194       96      0    0          0   

   Have0to0buy  Want0to0buy  Total  Income  
0            0          300    440   10000  
1            0         2529   2989       0  
2          590         1227   2681       0  
3            0           37    512       0  
4          100          219    774       0  


In [219]:
df['Yesterday payment'] = df['Total'].shift(1)
df['Today payment'] = df['Total'].shift(0)

df['Raise?'] = df['Today payment'] - df['Yesterday payment']

df['Up'] = df['Raise?'].apply(lambda x:1 if x>0 else 0)
print(df.head())

   Date  Breakfast  Lunch  Dinner  Dessert  Drink  Cat  Transport  \
0     1          0      0     125        0     15    0          0   
1     2        155      0     280       25      0    0          0   
2     3        270      0       0       53     40  425         76   
3     4        130      0     283       62      0    0          0   
4     5        165      0     194       96      0    0          0   

   Have0to0buy  Want0to0buy  Total  Income  Yesterday payment  Today payment  \
0            0          300    440   10000                NaN            440   
1            0         2529   2989       0              440.0           2989   
2          590         1227   2681       0             2989.0           2681   
3            0           37    512       0             2681.0            512   
4          100          219    774       0              512.0            774   

   Raise?  Up  
0     NaN   0  
1  2549.0   1  
2  -308.0   0  
3 -2169.0   0  
4   262.0   1  


In [220]:
df.dropna()
df = df.reset_index(drop=True)

print(df.head())

   Date  Breakfast  Lunch  Dinner  Dessert  Drink  Cat  Transport  \
0     1          0      0     125        0     15    0          0   
1     2        155      0     280       25      0    0          0   
2     3        270      0       0       53     40  425         76   
3     4        130      0     283       62      0    0          0   
4     5        165      0     194       96      0    0          0   

   Have0to0buy  Want0to0buy  Total  Income  Yesterday payment  Today payment  \
0            0          300    440   10000                NaN            440   
1            0         2529   2989       0              440.0           2989   
2          590         1227   2681       0             2989.0           2681   
3            0           37    512       0             2681.0            512   
4          100          219    774       0              512.0            774   

   Raise?  Up  
0     NaN   0  
1  2549.0   1  
2  -308.0   0  
3 -2169.0   0  
4   262.0   1  


In [221]:
df_normal = pd.DataFrame()

# 用比例學習（重複性高） -> 模型才學得到東西
df_normal['yesterday_payment_ratio'] = df['Total'].apply(pd.to_numeric) / df['Yesterday payment'].apply(pd.to_numeric)   # 數值比例

df_normal['target'] = df['Up'].shift(1)  # 預測漲or跌

df_normal = df_normal.dropna()
df_normal = df_normal.reset_index(drop=True)
print(df_normal)

print(df.head())

     yesterday_payment_ratio  target
0                   6.793182     0.0
1                   0.896956     1.0
2                   0.190974     0.0
3                   1.511719     0.0
4                   0.512920     1.0
..                       ...     ...
539                 1.240000     1.0
540                 0.821632     1.0
541                 0.715935     0.0
542                 1.032258     0.0
543                 0.468750     1.0

[544 rows x 2 columns]
   Date  Breakfast  Lunch  Dinner  Dessert  Drink  Cat  Transport  \
0     1          0      0     125        0     15    0          0   
1     2        155      0     280       25      0    0          0   
2     3        270      0       0       53     40  425         76   
3     4        130      0     283       62      0    0          0   
4     5        165      0     194       96      0    0          0   

   Have0to0buy  Want0to0buy  Total  Income  Yesterday payment  Today payment  \
0            0          300    440   

In [222]:
X = df_normal[['yesterday_payment_ratio']]
y = df_normal['target']

In [223]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

print(X_train)
print(X_test)

     yesterday_payment_ratio
289                 1.676119
245                 0.535311
339                 0.354063
457                 1.145078
300                 0.069108
..                       ...
129                 0.348411
144                 1.037946
72                  4.263975
235                 1.553191
37                  0.204082

[435 rows x 1 columns]
     yesterday_payment_ratio
244                 0.152850
132                 1.837288
511                 0.994413
133                 4.540590
482                 0.239819
..                       ...
187                10.517241
474                 0.368870
473                 1.542763
40                  0.063230
362                 0.166954

[109 rows x 1 columns]


In [224]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

In [225]:
from sklearn.svm import SVC
model = SVC(probability= True)  

model.fit(X_train, y_train)

X_test = scaler.transform(X_test)
y_pred = model.predict(X_test)

In [226]:
accuracy = accuracy_score(y_test, y_pred)
num_correct_samples = accuracy_score(y_test, y_pred, normalize=False)
con_matrix = confusion_matrix(y_test, y_pred)

print('number of correct sample: {}'.format(num_correct_samples))
print('accuracy: {}'.format(accuracy))
print('confusion matrix: {}'.format(con_matrix))
print('====================')

from sklearn.metrics import precision_score
print(f'Precision: {precision_score(y_test, y_pred)}')

from sklearn.metrics import recall_score
print(f'Recall: {recall_score(y_test, y_pred)}')

number of correct sample: 72
accuracy: 0.6605504587155964
confusion matrix: [[14 27]
 [10 58]]
Precision: 0.6823529411764706
Recall: 0.8529411764705882
